# Project 3 Part 1: Loading and Preprocessing

In [1]:
import pandas as pd
import numpy as np

# Load Data

In [2]:
basics_df = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t', low_memory=False)

In [3]:
akas_df = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep='\t', low_memory=False)

In [4]:
ratings_df = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep='\t', low_memory=False)

# Preprocessing basics_df

In [5]:
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


## Replace '/N'

In [6]:
basics_df.replace({'\\N':np.nan}, inplace=True)

## Eliminate entries null for runtimeMinutes or genre

In [61]:
basics_df.isna().sum()

tconst                  0
titleType               0
primaryTitle           11
originalTitle          11
isAdult                 1
startYear         1229118
endYear           9130315
runtimeMinutes    6753522
genres             427252
dtype: int64

In [7]:
basics_df.dropna(subset=['runtimeMinutes', 'genres'], inplace=True)

## Only keep if titleType==Movie

In [8]:
basics_df['titleType'].unique()

array(['short', 'movie', 'tvEpisode', 'tvSeries', 'tvShort', 'tvMovie',
       'tvMiniSeries', 'video', 'tvSpecial', 'videoGame'], dtype=object)

In [9]:
basics_df = basics_df[basics_df.titleType == 'movie']

## keep startYear 2000-2022

In [10]:
basics_df['startYear'] = pd.to_numeric(basics_df['startYear'])

In [11]:
basics_df['startYear'].dtype

dtype('float64')

In [12]:
year_filter = (basics_df['startYear'] > 1999) & (basics_df['startYear'] < 2023)

In [13]:
basics_df = basics_df[year_filter]

In [14]:
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
13079,tt0013274,movie,Istoriya grazhdanskoy voyny,Istoriya grazhdanskoy voyny,0,2021.0,NaN,133,Documentary
34792,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61094,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
66309,tt0067683,movie,Workers '71: Nothing About Us Without Us,Robotnicy 1971 - Nic o nas bez nas,0,2006.0,NaN,47,Documentary
67640,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama


## Eliminate movies that include "Documentary" in genre

In [15]:
is_documentary = basics_df['genres'].str.contains('documentary',case=False)
basics_df = basics_df[~is_documentary]

## Keep only US movies

In [16]:
akas_df.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [17]:
US_filter = akas_df['region'] == 'US'
akas_df = akas_df[US_filter]

In [18]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keep =basics_df['tconst'].isin(akas_df['titleId'])
basics_df = basics_df[keep]

In [19]:
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
34792,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
61094,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,NaN,70,Drama
67640,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
86771,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
93907,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [20]:
akas_df.loc[akas_df['region']=='US']

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,\N,imdbDisplay,\N,0
14,tt0000002,7,The Clown and His Dogs,US,\N,\N,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,\N,imdbDisplay,\N,0
36,tt0000005,1,Blacksmithing Scene,US,\N,alternative,\N,0
41,tt0000005,6,Blacksmith Scene #1,US,\N,alternative,\N,0
...,...,...,...,...,...,...,...,...
33301333,tt9916702,1,Loving London: The Playground,US,\N,\N,\N,0
33301370,tt9916720,10,The Demonic Nun,US,\N,tv,\N,0
33301372,tt9916720,12,The Nun 2,US,\N,imdbDisplay,\N,0
33301389,tt9916756,1,Pretty Pretty Black Girl,US,\N,imdbDisplay,\N,0


# Preprocesing akas_df

## keep only US movies
- see preprocessing basics_df

## Replace '/N'

In [21]:
akas_df.replace({'\\N':np.nan}, inplace=True)

## Only keep akas for movies in basics

In [23]:
# Filter the basics table down to only include the US by using the filter akas dataframe
akas_keep =akas_df['titleId'].isin(basics_df['tconst'])
akas_df = akas_df[akas_keep]

## Check Duplicates

In [35]:
akas_df.duplicated().sum()

0

In [39]:
akas_df.duplicated(subset=['titleId']).sum()

9533

In [42]:
akas_dup_df = akas_df[akas_df.duplicated(subset=['titleId'], keep=False)]

In [43]:
akas_dup_df.sort_values(by='titleId')

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
195744,tt0035423,35,Kate and Leopold,US,NaN,NaN,alternative spelling,0
195746,tt0035423,37,Kate & Leopold,US,NaN,imdbDisplay,NaN,0
709107,tt0088751,1,Attack of the B-Movie Monster,US,NaN,working,NaN,0
709110,tt0088751,4,The Naked Monster,US,NaN,imdbDisplay,NaN,0
922765,tt0113092,17,Final Encounter,US,NaN,dvd,NaN,0
...,...,...,...,...,...,...,...,...
33257920,tt9892604,4,Divine Decision: Double Down,US,NaN,NaN,new title,0
33265766,tt9896876,11,India Sweets and Spices,US,en,NaN,NaN,0
33265769,tt9896876,14,India Sweets and Spices,US,NaN,imdbDisplay,NaN,0
33297081,tt9913936,2,Diego's Paradise,US,NaN,alternative,NaN,0


# Preprocessing ratings_df

In [83]:
ratings_df.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1910
1,tt0000002,5.8,256
2,tt0000003,6.5,1714
3,tt0000004,5.6,169
4,tt0000005,6.2,2527


## Replace '/N'

In [27]:
ratings_df.replace({'\\N':np.nan}, inplace=True)

## Keep only US movies

In [28]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keep =ratings_df['tconst'].isin(akas_df['titleId'])
ratings_df = ratings_df[keep]

## Only keep akas for movies in basics

In [26]:
# Filter the basics table down to only include the US by using the filter akas dataframe
ratings_keep =ratings_df['tconst'].isin(basics_df['tconst'])
ratings_df = ratings_df[ratings_keep]

# Final info and saving

In [29]:
basics_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82697 entries, 34792 to 9255234
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          82697 non-null  object 
 1   titleType       82697 non-null  object 
 2   primaryTitle    82697 non-null  object 
 3   originalTitle   82697 non-null  object 
 4   isAdult         82697 non-null  object 
 5   startYear       82697 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  82697 non-null  object 
 8   genres          82697 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.3+ MB


In [30]:
akas_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92230 entries, 195744 to 33300906
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   titleId          92230 non-null  object
 1   ordering         92230 non-null  int64 
 2   title            92230 non-null  object
 3   region           92230 non-null  object
 4   language         938 non-null    object
 5   types            85124 non-null  object
 6   attributes       4250 non-null   object
 7   isOriginalTitle  92230 non-null  object
dtypes: int64(1), object(7)
memory usage: 6.3+ MB


In [31]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67942 entries, 17883 to 1261543
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         67942 non-null  object 
 1   averageRating  67942 non-null  float64
 2   numVotes       67942 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 2.1+ MB


In [47]:
basics_df.to_csv('Data/basics.csv.gz')

In [48]:
akas_df.to_csv('Data/akas.csv.gz')

In [49]:
ratings_df.to_csv('Data/ratings.csv.gz')